In [ ]:
import os
import numpy as np
import pandas as pd
from skimage import io, measure
import tifffile
from kneed import KneeLocator

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import plotly.io as pio
pio.renderers.default = 'jupyterlab'
import plotly.graph_objs as go

In [ ]:
def masks_to_labeled_masks(msk, labeled_cells):
    unq, inv = np.unique(msk.reshape(-1,), return_inverse=True)

    for i in unq:
        if i not in labeled_cells:
            unq[i] = 0

    labeled_msk = unq[inv].reshape(msk.shape)
    
    return labeled_msk

In [ ]:
import sys
sys.path.insert(0, '/u/home/f/f7xiesnm/project-zipursky/code/easifish-proc/bydatasets/')
from metadata_gene_chan import get_proj_metadata

meta = get_proj_metadata()

In [ ]:
ddir0 = "/u/home/f/f7xiesnm/project-zipursky/easifish/cdfv1_c1-2"
ddir1 = "/u/home/f/f7xiesnm/project-zipursky/easifish/results/viz_all_projections_jan29"
PLUS_ONE = False # False if table indices start from 1 ; True if from 0

# s3 resolution - to downsamp and rounding
f_msk  = ddir0 + '/outputs/r1/segmentation/r1-c3.tif'
f_tbl1 = ddir0 + '/proc/r12/spotcount.csv'
f_tbl2 = ddir0 + '/proc/r12/roi.csv'

f_img0 = ddir1 + '/cdfv1_c1-2_r1_autos1_flatfused_c0_s4.tiff'

metakey = 'cdfv1_c1-2'
channels = meta[metakey]['channels']
proj_targets = meta[metakey]['proj_targets']
colors = meta[metakey]['colors']

# S4 image shape
img_shape = io.imread(f_img0).shape
print(img_shape)

msk = io.imread(f_msk)
msk = np.array(msk)

msk = msk[::2, ::2, ::2]
msk = msk[:-1,:,:]
print(msk.shape)
assert np.all(msk.shape == img_shape)


# table
props = pd.read_csv(f_tbl1, index_col=0)
props2 = pd.read_csv(f_tbl2, index_col=0)
props = props.join(props2, how='left')
assert np.all(np.unique(msk[msk!=0]) == props.index.values)


In [ ]:
# # images 
# dataset_name = 'cdf04_c1-2_bino_r1_autos1_flatfused'
# channels = ['c0', 'c1', 'c2']

# all_props = []
# for ch in channels:
#     print(ch)
    
#     # image 
#     f_img = os.path.join(ddir1, f'{dataset_name}_{ch}_s4.tiff')

#     # view 1 image
#     img = io.imread(f_img)
#     img = np.array(img)

#     # measure intensity
#     props = measure.regionprops_table(msk.astype(int), 
#                                       intensity_image=img,
#                                       properties=['centroid', 'area', 'mean_intensity']) # , 'max_intensity', 'min_intensity'
#     props = pd.DataFrame(props)
#     print(props.shape)

#     all_props.append(props)

# check threshold (channel by channel)

In [ ]:
n = 2
log2ths = []

fig, axs = plt.subplots(1,n,figsize=(n*5,1*4))
for i, ch in enumerate(channels):
    color = colors[i]
    proj_target = proj_targets[i]
    
    ax = axs[0]
    # x y 
    intn = np.log2(1+props[ch].values)
    x  = np.sort(intn)
    y  = np.arange(len(intn))/len(intn)


    cond = y > 0.8
    kl = KneeLocator(x[cond], y[cond], curve="concave")

    x_elbow = kl.elbow
    y_elbow = kl.elbow_y
    log2ths.append(x_elbow)

    ax.plot(x, y, color=color)
    ax.plot(x_elbow, y_elbow, 'o', color=color)
    print(x_elbow, y_elbow)


    ax = axs[1]
    ax.plot(x[cond], y[cond], color=color, label=proj_target)
    ax.plot(x_elbow, y_elbow, 'o', color=color, )

    ax.legend()
    
log2ths = np.array(log2ths)
plt.show()

# visualize the masks

In [ ]:
print(log2ths)
thresholds = 2**log2ths-1 # np.array([5, 7, 5])
print(thresholds)

In [ ]:
# keep the masks with high intensity

for ch, th in zip(channels, thresholds):

    if PLUS_ONE:
        labeled_cells = 1+props[props[ch]>th].index.values #, props['area']<max_cellsize)].index.values
    else:
        labeled_cells = props[props[ch]>th].index.values #, props['area']<max_cellsize)].index.values
        
    labeled_masks = masks_to_labeled_masks(msk, labeled_cells)
    print(np.unique(labeled_masks).shape)

    # # save as tiff
    output = f_img0.replace('_c0_', f'_{ch}_').replace('.tiff', '_labeled_masks_countbased.tiff') 
    print(output)
    tifffile.imwrite(output, labeled_masks)
    


In [ ]:
# # download and viz

# rsync -av f7xiesnm@dtn.hoffman2.idre.ucla.edu:/u/home/f/f7xiesnm/project-zipursky/v1-bb/ms_reanalysis/240910 ~/Downloads/

# Visualize the dots 

In [ ]:
# plotly 3D

traces = []

allprops = props # .index.values # props['area']<max_cellsize)].index.values
allx = props['x'].values
ally = props['y'].values
allz = props['z'].values

trace = go.Scatter3d(x=allx, y=ally, z=allz, mode='markers',
                     marker=dict(size=2, color='lightgray', opacity=0.5))
traces.append(trace)

for i, (ch, th) in enumerate(zip(channels, thresholds)):
    labeled_props = props[props[ch]>th] # .index.values # props['area']<max_cellsize)].index.values
    x = labeled_props['x'].values
    y = labeled_props['y'].values
    z = labeled_props['z'].values
    color = colors[i]
    
    trace = go.Scatter3d(x=x, y=y, z=z, mode='markers',
                         marker=dict(size=3, color=color, opacity=0.5))
    traces.append(trace)
    
layout = go.Layout(title='',
                   scene=dict(
                       xaxis=dict(showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
                       yaxis=dict(showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
                       zaxis=dict(showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
                       xaxis_title='x', yaxis_title='y', zaxis_title='z',
                   ), 
                   scene_dragmode='orbit',
                  )

fig = go.Figure(data=traces, layout=layout)
fig.write_html("figure.html")
fig.show()

# quants and stats

In [ ]:
# bin spatial
stepsize_xy = 40
stepsize_z  = 20

xmax = allprops['x'].max()
ymax = allprops['y'].max()
zmax = allprops['z'].max()

xbins = np.arange(0, xmax+stepsize_xy, stepsize_xy)
ybins = np.arange(0, ymax+stepsize_xy, stepsize_xy)
zbins = np.arange(0, zmax+stepsize_z , stepsize_z)

# bin intensity (for binary label) 
for i, (ch, proj_target) in enumerate(zip(channels, proj_targets)): 
    allprops[proj_target] = (allprops[ch] > thresholds[i]).astype(int)

# along Z

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(2*5,1*4))
ax = axs[0]
sns.histplot(data=allprops['z'], bins=zbins, color='lightgray', 
             element='step', fill=False,
             label='cytoDAPI',
             ax=ax,
            )
sns.histplot(data=allprops.loc[np.sum(allprops[proj_targets], axis=1)>0, 'z'], bins=zbins, color='black',
             element='step', fill=False,
             label='projection labeled',
             ax=ax,
            )
ax.legend()
sns.despine(ax=ax)

ax = axs[1]
for i, target in enumerate(proj_targets):
    sns.histplot(data=allprops.loc[allprops[target]>0, 'z'], bins=zbins, color=colors[i], 
                 element='step', fill=False,
                 label=target,
                 ax=ax,
                )
ax.legend()
sns.despine(ax=ax)
plt.show()


# along XY

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(2*5,1*4))
ax = axs[0]
label = 'cytoDAPI labeled'
sns.histplot(data=allprops, 
             x='x', y='y', bins=[xbins, ybins], 
             element='step', fill=False,
             cmap='gray_r',
             cbar=True, cbar_kws={'shrink': 0.5},
             ax=ax,
            )
ax.set_aspect('equal')
sns.despine(ax=ax)
ax.set_title(label)

ax = axs[1]
label = 'projection labeled'
sns.histplot(data=allprops.loc[np.sum(allprops[proj_targets], axis=1)>0], 
             x='x', y='y', bins=[xbins, ybins], 
             element='step', fill=False,
             cmap='gray_r',
             cbar=True, cbar_kws={'shrink': 0.5},
             ax=ax,
            )
ax.set_aspect('equal')
sns.despine(ax=ax)
ax.set_title(label)
plt.show()

n = len(channels)
fig, axs = plt.subplots(1, n, figsize=(n*5,1*4))
for i, target in enumerate(proj_targets):
    ax = axs[i]
    color = colors[i]
    sns.histplot(data=allprops.loc[allprops[target]>0], 
                 x='x', y='y', bins=[xbins, ybins], 
                 element='step', fill=False,
                 cmap=f'{color[0].upper()}{color[1:]}s', 
                 cbar=True, cbar_kws={'shrink': 0.5},
                 ax=ax,
                )
    ax.set_aspect('equal')
    sns.despine(ax=ax)
    ax.set_title(target)
plt.show()